In [1]:
# Import Dependencies:
## Note: You may have to install a number of packages prior to running this notebook.
import numpy as np
import pandas as pd
import os
import glob
import pickle as pickle
import json
# import missingno
import itertools
from itertools import cycle
from datetime import datetime
from builtins import object
from datetime import date
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from concurrent import futures
from pandas import set_option
import ta
# import pandas_datareader.data as web
import warnings
import random
warnings.filterwarnings('ignore')

#visualization dependencies
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from plotly import graph_objects as go
from plotly import offline as py
import plotly.express as px
%matplotlib inline

#machine learning/statistical dependencies
import statsmodels.api as sm
from kneed import KneeLocator
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.manifold import TSNE
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA 
# from pykalman import KalmanFilter
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
import scipy.cluster.hierarchy as shc

import gc
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import dot, Dense, Reshape, Embedding
from tensorflow.keras.models import Sequential

from sklearn.metrics.pairwise import cosine_similarity
import faiss

#displaying
# set_option('precision', 3)

/tmp/ipykernel_18981/3803107103.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-21 03:32:34.501944: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 03:32:34.539260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AV

In [2]:
tqdm.pandas()

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)


# Read Data

In [4]:
df = pd.read_csv('/home/ec2-user/Capstone/1999-12-01-2023-12-31.csv')

In [5]:
spy_df = pd.read_csv('/home/ec2-user/Capstone/1999-12-01-2023-12-31_SPY.csv')

In [6]:
df.shape

(2701713, 10)

In [7]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,1999-12-01,47.344,47.625,46.781,47.219,24.405,2062600.000,MMM,Industrials,Industrial Conglomerates
1,1999-12-02,47.500,49.219,47.469,48.594,25.116,2617400.000,MMM,Industrials,Industrial Conglomerates
2,1999-12-03,48.750,50.406,48.750,49.656,25.665,3060600.000,MMM,Industrials,Industrial Conglomerates
3,1999-12-06,49.250,49.531,48.688,48.906,25.277,1282200.000,MMM,Industrials,Industrial Conglomerates
4,1999-12-07,48.750,48.750,46.500,46.500,24.034,2190400.000,MMM,Industrials,Industrial Conglomerates


# Feature Engineering

In [8]:
class feature_engineering(object):
    def __init__(self):
        super(feature_engineering, self).__init__()

    def generate_technical_indicator(self, data_df): 
        '''
        Function to generate additional technical indicators for the stock

        Uses the "statsmodels.tsa.stattools" (as "ts") package to apply 
        the equations specified in the "Technical Indicators" markdown section 
        to stock closing data.
        
        Input:
        data_df-- Dataframe containing stock finacials data
        
        Output:
        Stock finacials data with added Dataframe of feature obtained from feature engineering
        ''' 
        # 1. Momentum Indicators
        # Relative Strength Index
        df = data_df
        df['rsi'] = ta.momentum.rsi(df['Close'], window=14)
        # Kaufman’s Adaptive Moving Average (KAMA)
        df['kama'] = ta.momentum.kama(df['Close'],window=14)

        # 2. Volume Indicators
        # Accumulation/Distribution Index (ADI)
        df['adi'] = ta.volume.acc_dist_index(df['High'], df['Low'], df['Close'], df['Volume'])

        # Volume-price trend (VPT)
        df['vpt'] = ta.volume.volume_price_trend(df['Close'], df['Volume'])

        # 3. Volatility Indicators
        # Average True Range (ATR)
        df['atr'] = ta.volatility.average_true_range(df['High'], df['Low'],df['Close'], window=14)

        # Bollinger Bands (BB) N-period simple moving average (MA)
        df['bb_ma'] = ta.volatility.bollinger_mavg(df['Close'], window=20)

        # 4. Trend Indicators
        # Average Directional Movement Index (ADX)
        df['adx'] = ta.trend.adx(df['High'], df['Low'], df['Close'], window=14)

        # Exponential Moving Average
        df['ema'] = ta.trend.ema_indicator(df['Close'], window=14)

        # Moving Average Convergence Divergence (MACD)
        df['macd'] = ta.trend.macd(df['Close'], window_fast=14, window_slow=30)

        # 5. Other Indicators
        # Daily Log Return (DLR)
        df['dlr'] = ta.others.daily_log_return(df['Close'])

        # Daily Returns
        df['daily_returns'] = df['Adj Close'].pct_change()

        # Moving Averages
        averages = [50,200]
        for avg in averages:
            col_name = str(avg) +' Days Average'
            df[col_name] = df['Adj Close'].rolling(window = avg, center = False).mean()

        return df

In [9]:
# Create an instance of the feature_engineering class object
fe_obj = feature_engineering()

# Add technical indicators to combined_data
df = fe_obj.generate_technical_indicator(df)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry,...,vpt,atr,bb_ma,adx,ema,macd,dlr,daily_returns,50 Days Average,200 Days Average
0,1999-12-01,47.344,47.625,46.781,47.219,24.405,2062600.000,MMM,Industrials,Industrial Conglomerates,...,NaN,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-12-02,47.500,49.219,47.469,48.594,25.116,2617400.000,MMM,Industrials,Industrial Conglomerates,...,76218.134,0.000,NaN,0.000,NaN,NaN,2.870,0.029,NaN,NaN
2,1999-12-03,48.750,50.406,48.750,49.656,25.665,3060600.000,MMM,Industrials,Industrial Conglomerates,...,143138.005,0.000,NaN,0.000,NaN,NaN,2.163,0.022,NaN,NaN
3,1999-12-06,49.250,49.531,48.688,48.906,25.277,1282200.000,MMM,Industrials,Industrial Conglomerates,...,123771.863,0.000,NaN,0.000,NaN,NaN,-1.522,-0.015,NaN,NaN
4,1999-12-07,48.750,48.750,46.500,46.500,24.034,2190400.000,MMM,Industrials,Industrial Conglomerates,...,16001.384,0.000,NaN,0.000,NaN,NaN,-5.045,-0.049,NaN,NaN


In [10]:
spy_df = fe_obj.generate_technical_indicator(spy_df)

# Filter Data and Select Date Range and Columns

In [11]:
input_data = df[df.Date!=df.Date.min()][['Date','Ticker','daily_returns']].dropna()
input_data = input_data[input_data.Date.between('2011-01-01','2016-01-01')]
input_data = input_data.groupby('Ticker').progress_apply(lambda group: group.iloc[1:,]).reset_index(drop=True)
input_data.head()

  0%|          | 0/481 [00:00<?, ?it/s]

,Date,Ticker,daily_returns
0,2011-01-04,A,-0.009
1,2011-01-05,A,-0.002
2,2011-01-06,A,0.002
3,2011-01-07,A,0.003
4,2011-01-10,A,0.014


In [12]:
spy_df['spy_daily_returns'] = spy_df['daily_returns']

In [13]:
print(input_data.shape)
input_data = pd.merge(input_data,spy_df[['Date','spy_daily_returns']],on='Date',how='left')
print(input_data.shape)


(582690, 3)
(582690, 4)


In [14]:
input_data['spy_daily_returns'].abs().describe(np.arange(0,1,0.1))

count   582690.000
mean         0.007
std          0.007
min          0.000
0%           0.000
10%          0.001
20%          0.002
30%          0.003
40%          0.004
50%          0.005
60%          0.006
70%          0.008
80%          0.011
90%          0.015
max          0.065
Name: spy_daily_returns, dtype: float64

In [15]:
# input_data['daily_returns'] = np.where(input_data['spy_daily_returns'].abs() <= 0.005,\
#                                        input_data['daily_returns'],input_data['daily_returns']-input_data['spy_daily_returns'])
input_data['daily_returns'] = input_data['daily_returns']-input_data['spy_daily_returns']
print(input_data.shape)
input_data = input_data[input_data['spy_daily_returns'].abs() > 0.005]
print(input_data.shape)

(582690, 4)
(281416, 4)


In [16]:
input_data.daily_returns.abs().describe(np.arange(0,0.9,0.1).tolist()+np.arange(0.9,1,0.01).tolist())

count   281416.000
mean         0.010
std          0.011
min          0.000
0%           0.000
10%          0.001
20%          0.002
30%          0.004
40%          0.005
50%          0.007
60%          0.009
70%          0.011
80%          0.015
90%          0.021
91%          0.022
92%          0.023
93%          0.025
94%          0.026
95%          0.028
96%          0.031
97%          0.035
98%          0.040
99%          0.052
max          0.610
Name: daily_returns, dtype: float64

In [17]:
input_data.sort_values('daily_returns',ascending=False).head()

,Date,Ticker,daily_returns,spy_daily_returns
546143,2013-04-19,VRTX,0.610,0.009
274137,2012-01-25,ILMN,0.455,0.008
546440,2014-06-24,VRTX,0.410,-0.006
185949,2015-12-16,ENPH,0.371,0.015
509508,2012-07-26,TMUS,0.351,0.016


In [18]:
# df[df.Ticker=='WMB'][input_data.columns.tolist()+['Adj Close']].set_index('Date').loc['2002-07-26':]

In [19]:
print(input_data.shape)
input_data = input_data[input_data.daily_returns.abs()<0.5]
input_data.shape

(281416, 4)


(281415, 4)

# Prepare Data for Stock2Vec

In [20]:
all_dates = sorted(input_data.Date.unique())
all_dates

['2011-01-05',
 '2011-01-12',
 '2011-01-14',
 '2011-01-19',
 '2011-01-24',
 '2011-01-28',
 '2011-01-31',
 '2011-02-01',
 '2011-02-07',
 '2011-02-11',
 '2011-02-16',
 '2011-02-22',
 '2011-02-23',
 '2011-02-25',
 '2011-02-28',
 '2011-03-01',
 '2011-03-03',
 '2011-03-04',
 '2011-03-07',
 '2011-03-08',
 '2011-03-10',
 '2011-03-11',
 '2011-03-14',
 '2011-03-15',
 '2011-03-16',
 '2011-03-17',
 '2011-03-21',
 '2011-03-24',
 '2011-03-29',
 '2011-03-30',
 '2011-04-12',
 '2011-04-18',
 '2011-04-19',
 '2011-04-20',
 '2011-04-21',
 '2011-04-26',
 '2011-04-27',
 '2011-05-04',
 '2011-05-05',
 '2011-05-10',
 '2011-05-11',
 '2011-05-13',
 '2011-05-16',
 '2011-05-18',
 '2011-05-20',
 '2011-05-23',
 '2011-05-31',
 '2011-06-01',
 '2011-06-03',
 '2011-06-06',
 '2011-06-09',
 '2011-06-10',
 '2011-06-14',
 '2011-06-15',
 '2011-06-20',
 '2011-06-21',
 '2011-06-22',
 '2011-06-24',
 '2011-06-27',
 '2011-06-28',
 '2011-06-29',
 '2011-06-30',
 '2011-07-01',
 '2011-07-07',
 '2011-07-08',
 '2011-07-11',
 '2011-07-

In [21]:
all_sentences_returns = []
all_sentences_tickers = []
for d in tqdm(all_dates):
    sub_df = input_data[input_data.Date == d]
    sub_df = sub_df.sort_values('daily_returns',ascending=True)
    all_sentences_returns.append(sub_df['daily_returns'])
    all_sentences_tickers.append(' '.join(sub_df['Ticker']))

  0%|          | 0/608 [00:00<?, ?it/s]

In [22]:
all_sentences_returns[0]

159877   -0.054
325218   -0.049
352525   -0.031
554368   -0.026
541798   -0.026
          ...  
74025     0.054
398045    0.057
13326     0.065
19611     0.068
394253    0.072
Name: daily_returns, Length: 445, dtype: float64

In [23]:
np.savetxt('./stock_str.txt',all_sentences_tickers,delimiter=" ",newline='\n',fmt='%s')

In [24]:
text_ds = tf.data.TextLineDataset('./stock_str.txt').filter(lambda x: tf.cast(tf.strings.length(x),bool))

2024-02-21 03:33:18.477109: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [25]:
def custom_standardization(t):
    return tf.strings.upper(t)

vocab_size = input_data.Ticker.nunique()
print("vocab_size: ",vocab_size)
sequence_length = max([len(l.split(' ')) for l in all_sentences_tickers])
print("sequence_length: ",sequence_length)
vectorize_layer = layers.experimental.preprocessing.TextVectorization(
        standardize = custom_standardization,
        max_tokens = vocab_size,
        output_mode = 'int',
        output_sequence_length = sequence_length
)

vocab_size:  481
sequence_length:  481


In [26]:
vectorize_layer.adapt(text_ds.batch(sequence_length))
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'ZION', 'ZBRA', 'ZBH', 'YUM', 'XRAY', 'XOM', 'XEL', 'WYNN', 'WY', 'WTW', 'WST', 'WRB', 'WMT', 'WMB', 'WM', 'WHR', 'WFC', 'WELL']


In [27]:
text_vector_ds = text_ds.batch(sequence_length).prefetch(tf.data.experimental.AUTOTUNE).map(vectorize_layer).unbatch()

In [28]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
print(input_data.Date.nunique())

608
608


In [29]:
for seq in sequences:
    print(seq[:20],seq[-20:])

[322 197 176  22  31  98 347 293 357  60  63 394 247 155 131 213 433 423
 328 137] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[407 322 255 143 404 103 432 335 356 139  12 392 171 194   2 163 430 287
 424 233] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[430 371  77   5 155  50 418 161 202  90 268 392 168 399 146 397  91  86
 442 234] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[170 353  50 367 103 147 426  41 102 180 258  77 445 387 288 314 400  76
 430 192] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[342 282 372 430 182 400   2  62 235 215 278 273   4 254 173 200  39 351
 356 328] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[288 444 417 387  98 143 264 179 265 291 338 134 188 180 195  78 171  36
  51  72] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[264  17 144 385 444 122 227 328 371 288 342 154 198  57  43 335 125 229
 206 105] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[132  80  28  98 250 304 343 235  93 205 120 364  50 275  86 406 379 154
  43 255] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## generate_training_data

In [30]:
training_data_map = {}

In [31]:
def generate_training_data(sequences):
    for i, sequence in tqdm(enumerate(sequences)):
        sequence = sequence[sequence != 0]
        
        curr_return_list = all_sentences_returns[i].values.tolist()
        
        all_combinations_iter = itertools.combinations(range(len(sequence)),2)
        
        for j in tqdm(all_combinations_iter):
            target_index, context_index = j
            
            target_stock = sequence[target_index]
            context_stock = sequence[context_index]

            if target_stock == context_stock:
                continue
            target_stock_return = curr_return_list[target_index]
            context_stock_return = curr_return_list[context_index]
            
#             print(inverse_vocab[target_stock], inverse_vocab[context_stock], target_stock_return,context_stock_return)

            label = abs(target_stock_return-context_stock_return)*100.0
            
            key = (target_stock, context_stock) if target_stock < context_stock else (context_stock, target_stock)
            
            if key in training_data_map:
                new_avg = (training_data_map[key][0]*training_data_map[key][1]+label)/(training_data_map[key][1]+1)
                training_data_map[key] = [new_avg, training_data_map[key][1]+1]
            else:
                training_data_map[key] = [label, 1]
            
#             break
#         break

In [32]:
all_returns = []
all_tickers = []
for i, sequence in tqdm(enumerate(sequences[0:1])):
    sequence = sequence[sequence != 0]
    sequence_tickers = [inverse_vocab[s] for s in sequence]
    
    curr_return_list = all_sentences_returns[i].values.tolist()
    
    all_returns.append(curr_return_list)
    all_tickers.append(sequence_tickers)
    


0it [00:00, ?it/s]

In [33]:
def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

In [34]:
flat_all_returns = flatten_comprehension(all_returns)
flat_all_tickers = flatten_comprehension(all_tickers)
ticker_level_df = pd.DataFrame({'return':[f*100 for f in flat_all_returns],'Ticker':flat_all_tickers})
# ticker_level_df = ticker_level_df.groupby('ticker')['return']
ticker_level_df.head()

,return,Ticker
0,-5.379,DLTR
1,-4.871,LRCX
2,-3.107,MLM
3,-2.644,WBD
4,-2.608,VMC


In [35]:
generate_training_data(sequences)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [36]:
targets = [c[0] for c in tqdm([*training_data_map.keys()])]
contexts = [c[1] for c in tqdm([*training_data_map.keys()])]
labels = [training_data_map[c][0] for c in tqdm([*training_data_map.keys()])]


  0%|          | 0/114960 [00:00<?, ?it/s]

  0%|          | 0/114960 [00:00<?, ?it/s]

  0%|          | 0/114960 [00:00<?, ?it/s]

In [37]:
# Count distribution
pd.Series(training_data_map[c][1] for c in tqdm([*training_data_map.keys()])).describe(np.arange(0,1,0.1))

  0%|          | 0/114960 [00:00<?, ?it/s]

count   114960.000
mean       565.637
std        119.401
min         70.000
0%          70.000
10%        412.000
20%        608.000
30%        608.000
40%        608.000
50%        608.000
60%        608.000
70%        608.000
80%        608.000
90%        608.000
max        608.000
dtype: float64

In [38]:
# Average diff distribution
pd.Series(training_data_map[c][0] for c in tqdm([*training_data_map.keys()])).describe(np.arange(0,1,0.1))

  0%|          | 0/114960 [00:00<?, ?it/s]

count   114960.000
mean         1.452
std          0.415
min          0.065
0%           0.065
10%          1.020
20%          1.122
30%          1.205
40%          1.284
50%          1.370
60%          1.468
70%          1.587
80%          1.742
90%          2.014
max          5.256
dtype: float64

In [39]:
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

In [40]:
targets_train, targets_test, contexts_train, contexts_test, labels_train, labels_test = train_test_split(
        targets, contexts, labels, test_size = 0.2, shuffle=True, random_state=42
)

# Create Model

In [56]:
EMBEDDING_DIM = 32

target_model = Sequential()
target_model.add(Embedding(vocab_size, EMBEDDING_DIM, embeddings_initializer='he_normal',input_length=1,name='target'))
target_model.add(Reshape((EMBEDDING_DIM, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, EMBEDDING_DIM, embeddings_initializer='he_normal',input_length=1,name='context'))
context_model.add(Reshape((EMBEDDING_DIM, )))

dot_product = dot([target_model.output, context_model.output],axes=1,normalize=True)
dot_product = Dense(1,activation='relu')(dot_product)

model = tf.keras.Model(inputs=[target_model.input, context_model.input], outputs=dot_product)

optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss='mse',optimizer = optimizer, metrics = ['mae'],run_eagerly=True)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 target_input (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 context_input (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 target (Embedding)          (None, 1, 32)                15392     ['target_input[0][0]']        
                                                                                                  
 context (Embedding)         (None, 1, 32)                15392     ['context_input[0][0]']       
                                                                                            

# Train model

In [57]:
model.fit([targets_train,contexts_train], labels_train,\
          validation_data = ([targets_test, contexts_test], labels_test),\
          batch_size = 128,\
          epochs = 50)

Epoch 1/50
719/719 [==============================] - 10s 13ms/step - loss: 0.9846 - mae: 0.8006 - val_loss: 0.1063 - val_mae: 0.2322
Epoch 2/50
719/719 [==============================] - 10s 13ms/step - loss: 0.0795 - mae: 0.1960 - val_loss: 0.0582 - val_mae: 0.1636
Epoch 3/50
719/719 [==============================] - 9s 13ms/step - loss: 0.0379 - mae: 0.1226 - val_loss: 0.0269 - val_mae: 0.1015
Epoch 4/50
719/719 [==============================] - 9s 13ms/step - loss: 0.0179 - mae: 0.0752 - val_loss: 0.0151 - val_mae: 0.0710
Epoch 5/50
719/719 [==============================] - 9s 13ms/step - loss: 0.0112 - mae: 0.0584 - val_loss: 0.0110 - val_mae: 0.0611
Epoch 6/50
719/719 [==============================] - 10s 13ms/step - loss: 0.0084 - mae: 0.0537 - val_loss: 0.0087 - val_mae: 0.0564
Epoch 7/50
719/719 [==============================] - 10s 13ms/step - loss: 0.0070 - mae: 0.0529 - val_loss: 0.0077 - val_mae: 0.0570
Epoch 8/50
719/719 [==============================] - 9s 13ms/ste

# Retrieve Embeddings

In [58]:
def get_embeddings(df_to_parse):
    weights = model.get_layer('target').get_weights()[0]
    return np.array([weights[inverse_vocab.index(stock)] if stock in inverse_vocab else [np.nan]*EMBEDDING_DIM\
                     for stock in tqdm(df_to_parse.Ticker.unique())]) 

In [59]:
embedding_values = pd.DataFrame(get_embeddings(input_data),columns=['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)])
embedding_values['Ticker'] = input_data.Ticker.unique()
print(embedding_values.shape)
embedding_values = pd.merge(embedding_values, df[['Ticker','GICS Sector']].drop_duplicates(),on='Ticker',how='left')
embedding_values = embedding_values.dropna()
print(embedding_values.shape)
embedding_values

  0%|          | 0/481 [00:00<?, ?it/s]

(481, 33)
(479, 34)


,STOCK2VEC_0,STOCK2VEC_1,STOCK2VEC_2,STOCK2VEC_3,STOCK2VEC_4,STOCK2VEC_5,STOCK2VEC_6,STOCK2VEC_7,STOCK2VEC_8,STOCK2VEC_9,...,STOCK2VEC_24,STOCK2VEC_25,STOCK2VEC_26,STOCK2VEC_27,STOCK2VEC_28,STOCK2VEC_29,STOCK2VEC_30,STOCK2VEC_31,Ticker,GICS Sector
0,-0.012,-0.037,-0.031,0.015,-0.033,0.155,0.089,0.051,-0.096,0.099,...,0.178,0.656,0.161,0.213,0.059,0.322,-0.557,0.055,A,Health Care
1,0.093,-0.027,-0.130,-0.040,0.020,-0.015,0.158,-0.083,-0.058,-0.009,...,0.512,0.342,-0.026,0.228,-0.020,0.075,-0.621,0.125,AAL,Industrials
2,0.094,-0.137,-0.063,0.121,-0.072,0.156,0.267,0.058,-0.031,-0.088,...,0.568,0.610,0.053,0.086,-0.115,0.292,-0.183,0.226,AAPL,Information Technology
3,-0.028,0.012,-0.070,0.031,-0.027,0.147,0.135,0.070,-0.147,-0.014,...,0.386,0.302,0.029,0.136,-0.026,0.416,-0.708,0.014,ABBV,Health Care
4,0.002,-0.080,-0.037,0.139,0.026,0.119,0.099,0.227,-0.098,0.084,...,0.571,0.402,0.079,0.001,0.008,0.451,-0.378,0.261,ABT,Health Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,0.018,-0.045,-0.004,0.028,-0.071,0.154,0.079,0.232,-0.032,-0.031,...,0.321,1.017,-0.012,-0.030,-0.054,0.207,-0.270,0.137,YUM,Consumer Discretionary
477,-0.027,0.012,-0.028,0.126,-0.033,0.105,0.059,0.201,-0.118,-0.020,...,0.398,0.909,0.103,-0.049,-0.066,0.297,-0.234,0.154,ZBH,Health Care
478,-0.038,-0.008,0.024,0.010,0.006,0.066,0.116,0.074,-0.120,-0.089,...,0.474,0.959,0.081,-0.010,-0.010,0.323,-0.259,0.054,ZBRA,Information Technology
479,-0.032,0.088,-0.055,0.057,0.007,0.153,0.137,-0.142,0.018,0.178,...,0.227,0.874,0.107,-0.006,-0.061,0.335,-0.271,-0.026,ZION,Financials


# Plot Stocks

In [71]:
from sklearn.decomposition import PCA
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

def plot_3D(result,embedding_values,ticker_to_highlight,color_column):
    PCs_3d = pd.DataFrame(result)
    PCs_3d.columns = ['PC1','PC2','PC3']
    
    plot_df = pd.concat([PCs_3d.reset_index(drop=True),embedding_values.reset_index(drop=True)],axis=1)
    plot_df = pd.merge(plot_df, ticker_level_df,on='Ticker',how='left')
    print(plot_df.shape)
    plot_df = plot_df.dropna()
    print(plot_df.shape)

    print(plot_df.columns)
    columns_to_hover = ['Ticker','return']
    
    all_stocks = px.scatter_3d(
            plot_df,
            x = 'PC1',
            y = 'PC2',
            z = 'PC3',
            title = 'Stock2Vec',
#             color = 'GICS Sector',
#             color = 'return',
            color = color_column,
            custom_data = columns_to_hover
    )
    
    all_stocks.update_traces(marker={'size':5})
    all_stocks.update_traces(hovertemplate="<br>".join(['\"'+f+": %{customdata["+str(i)+']}\"' \
                                                    for i, f in enumerate(columns_to_hover)]))
    
    search_stock = plot_df[plot_df['Ticker']==ticker_to_highlight].iloc[0:1]
    
    search_stock_trace = px.scatter_3d(
                                search_stock,
                                x = [search_stock['PC1'].iloc[0]],
                                y = [search_stock['PC2'].iloc[0]],
                                z = [search_stock['PC3'].iloc[0]],
                                symbol_sequence = ['diamond'],
                                size = [7],
                                color_discrete_sequence = ['red'],
                                custom_data = columns_to_hover
    )
    
    search_stock_trace.update_traces(hovertemplate="<br>".join(['\"'+f+": %{customdata["+str(i)+']}\"' \
                                                    for i, f in enumerate(columns_to_hover)]))
    
    fig = go.Figure(data=all_stocks.data+search_stock_trace.data)
    fig.show()

all_stock_vecs = embedding_values.drop(['Ticker','GICS Sector'],axis=1)
pca = PCA(n_components=3)
result = pca.fit_transform(all_stock_vecs)

print(pca.explained_variance_ratio_.cumsum())
print(result.shape, embedding_values.shape)


[0.20261912 0.36390838 0.4495577 ]
(479, 3) (479, 34)


In [72]:
plot_3D(result, embedding_values,'MSFT','return')

(479, 38)
(445, 38)
Index(['PC1', 'PC2', 'PC3', 'STOCK2VEC_0', 'STOCK2VEC_1', 'STOCK2VEC_2',
       'STOCK2VEC_3', 'STOCK2VEC_4', 'STOCK2VEC_5', 'STOCK2VEC_6',
       'STOCK2VEC_7', 'STOCK2VEC_8', 'STOCK2VEC_9', 'STOCK2VEC_10',
       'STOCK2VEC_11', 'STOCK2VEC_12', 'STOCK2VEC_13', 'STOCK2VEC_14',
       'STOCK2VEC_15', 'STOCK2VEC_16', 'STOCK2VEC_17', 'STOCK2VEC_18',
       'STOCK2VEC_19', 'STOCK2VEC_20', 'STOCK2VEC_21', 'STOCK2VEC_22',
       'STOCK2VEC_23', 'STOCK2VEC_24', 'STOCK2VEC_25', 'STOCK2VEC_26',
       'STOCK2VEC_27', 'STOCK2VEC_28', 'STOCK2VEC_29', 'STOCK2VEC_30',
       'STOCK2VEC_31', 'Ticker', 'GICS Sector', 'return'],
      dtype='object')


In [73]:
plot_3D(result, embedding_values,'MSFT','GICS Sector')

(479, 38)
(445, 38)
Index(['PC1', 'PC2', 'PC3', 'STOCK2VEC_0', 'STOCK2VEC_1', 'STOCK2VEC_2',
       'STOCK2VEC_3', 'STOCK2VEC_4', 'STOCK2VEC_5', 'STOCK2VEC_6',
       'STOCK2VEC_7', 'STOCK2VEC_8', 'STOCK2VEC_9', 'STOCK2VEC_10',
       'STOCK2VEC_11', 'STOCK2VEC_12', 'STOCK2VEC_13', 'STOCK2VEC_14',
       'STOCK2VEC_15', 'STOCK2VEC_16', 'STOCK2VEC_17', 'STOCK2VEC_18',
       'STOCK2VEC_19', 'STOCK2VEC_20', 'STOCK2VEC_21', 'STOCK2VEC_22',
       'STOCK2VEC_23', 'STOCK2VEC_24', 'STOCK2VEC_25', 'STOCK2VEC_26',
       'STOCK2VEC_27', 'STOCK2VEC_28', 'STOCK2VEC_29', 'STOCK2VEC_30',
       'STOCK2VEC_31', 'Ticker', 'GICS Sector', 'return'],
      dtype='object')


# Adhoc Check

In [61]:
p1 = embedding_values[embedding_values.Ticker=='KO'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='PEP'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.99178224]])

In [62]:
p1 = embedding_values[embedding_values.Ticker=='KO'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='TSLA'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.71193358]])

In [63]:
p1 = embedding_values[embedding_values.Ticker=='XOM'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='CVX'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.98484866]])

In [64]:
p1 = embedding_values[embedding_values.Ticker=='HD'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='LOW'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.98293675]])

In [65]:
p1 = embedding_values[embedding_values.Ticker=='LOW'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='META'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.70446886]])

In [66]:
p1 = embedding_values[embedding_values.Ticker=='GM'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='F'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.96806006]])

In [67]:
p1 = embedding_values[embedding_values.Ticker=='ABBV'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]
p2 = embedding_values[embedding_values.Ticker=='JNJ'][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]]

cosine_similarity(p1,p2)

array([[0.59700976]])

# Top 10 Similar stocks for the prompted stock is

In [77]:
stock_to_check = 'ABBV'
left = np.ascontiguousarray(embedding_values[embedding_values.Ticker==stock_to_check][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]],np.float32)

right = np.ascontiguousarray(embedding_values[embedding_values.Ticker!=stock_to_check][['STOCK2VEC_'+str(i) for i in range(EMBEDDING_DIM)]],np.float32)

index = faiss.IndexFlatIP(EMBEDDING_DIM)
faiss.normalize_L2(right)
index.add(right)

faiss.normalize_L2(left)

distances, neighbors = index.search(left, k = 10)
print(distances)
print(neighbors)
print([inverse_vocab[c] for c in neighbors[0]])

[[0.90299606 0.88975096 0.8810214  0.8807285  0.875248   0.8746159
  0.8518589  0.8492214  0.84733075 0.8316542 ]]
[[230 477  98 209  10  14 218 293 224   0]]
['IRM', 'QRVO', 'RMD', 'KR', 'WY', 'WMT', 'JPM', 'EW', 'JBHT', '']
